<a href="https://colab.research.google.com/github/crneubert/best-music/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Run this whole cell
%cd /content
!rm -rf best-music
!git clone https://github.com/crneubert/best-music.git
%cd best-music

# Set your Git identity (required for commits)
!git config user.name "crneubert"
!git config user.email "you@example.com"   # use any email; GitHub shows it if it's public

# Create main (repo is empty → no branches yet)
!git checkout -b main

# Add starter files
!mkdir -p notebooks
!printf "# best-music\n" > README.md
!printf "*.ipynb_checkpoints/\n" > .gitignore
!touch notebooks/.gitkeep

# Commit locally
!git add .
!git commit -m "Initial commit from Colab"

import os
os.environ["GH_TOKEN"] = "git@github.com:crneubert/best-music.git"

# Set remote to use the token for this session and push 'main'
!git remote set-url origin https://$GH_TOKEN@github.com/crneubert/best-music.git
!git push -u origin main


/content
Cloning into 'best-music'...
/content/best-music
Switched to a new branch 'main'
[main (root-commit) 522fc44] Initial commit from Colab
 3 files changed, 2 insertions(+)
 create mode 100644 .gitignore
 create mode 100644 README.md
 create mode 100644 notebooks/.gitkeep
fatal: unable to access 'https://github.com:crneubert/best-music.git@github.com/crneubert/best-music.git/': URL using bad/illegal format or missing URL
